In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Marie Curie received the Nobel Prize in Physics")
options = {"collapse_phrases": True}

# we can display
spacy.displacy.render(doc, style="dep", options=options, jupyter=True)
"""
(verb: receive, subject: p:Person, object: a:Prize) → (relationship: RECEIVE_PRIZE, from: p, to:a)

Here, “receive” (considered in the lemmatized version) is an English verb, whereas RECEIVE_PRIZE is a relationship type
(a semantic relation).
"""

In [ ]:
import os

from py2neo import Graph

# Connect to database
url = "bolt://localhost:7687"
username = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD_")
database = "spacy-ner"

graph = Graph(url, auth=(username, password), name=database)

In [ ]:
query = """
MATCH (verb:TagOccurrence {pos: "VBD", lemma:"receive"})
WITH verb
MATCH p=(verb)-[:IS_DEPENDENT {type:"nsubj"}]->(subject)-[:PARTICIPATES_IN]-> (person:NamedEntity {type: "PERSON"})
MATCH q=(verb)-[:IS_DEPENDENT {type:"dobj"}]->(object)-[:PARTICIPATES_IN]-> (woa:NamedEntity {type: "WORK_OF_ART"})
RETURN verb, person, woa, p, q
"""

sentence = "Marie Curie received the Nobel Prize in Physics in 1903."
"""
 But “receive” conveys many relations, and we don’t want instances of “receive”
 involving other types of arguments to be translated into the RECEIVE_PRIZE relationship

 “The Committee awarded the prize to Marie Curie.”
 “The prize was awarded to Marie Curie.”

 (relationship: "win", subject: p:Person, object: a:Prize) ®
  (relationship: RECEIVE_PRIZE, from: p, to:a)
 (relationship: "award", indirect-object: p:Person, object: a:Prize) ®
  (relationship: RECEIVE_PRIZE, from: p, to:a)
"""
res = graph.run(query).to_data_frame()
res

In [ ]:
query = """
MATCH (nodelPrize:Entity {type:"WORK_OF_ART"})<-[:TO]-(rel:Relationship {type: "RECEIVE_PRIZE"})-[:FROM]->(winner:Entity {type: "PERSON"})
WHERE nodelPrize.id CONTAINS "the Nobel Prize in Physics"
RETURN winner
"""
res = graph.run(query).to_data_frame()
res

## ConceptNet

In [4]:
import requests

obj = requests.get("http://api.conceptnet.io/c/en/marie_curie").json()
print(obj['edges'][0]['rel']['label'] + ": " + obj['edges'][0]['end']['label'])

Synonym: Marya Sklodowska


In [9]:
from pprint import pprint

obj = requests.get("http://api.conceptnet.io/c/en/blueprint").json()
pprint(obj)

{'@context': ['http://api.conceptnet.io/ld/conceptnet5.7/context.ld.json'],
 '@id': '/c/en/blueprint',
 'edges': [{'@id': '/a/[/r/Synonym/,/c/pt/planta/n/wn/artifact/,/c/en/blueprint/n/wn/artifact/]',
            '@type': 'Edge',
            'dataset': '/d/wordnet/3.1',
            'end': {'@id': '/c/en/blueprint/n/wn/artifact',
                    '@type': 'Node',
                    'label': 'blueprint',
                    'language': 'en',
                    'sense_label': 'n, artifact',
                    'term': '/c/en/blueprint'},
            'license': 'cc:by/4.0',
            'rel': {'@id': '/r/Synonym',
                    '@type': 'Relation',
                    'label': 'Synonym'},
            'sources': [{'@id': '/s/resource/wordnet/rdf/3.1',
                         '@type': 'Source',
                         'contributor': '/s/resource/wordnet/rdf/3.1'}],
            'start': {'@id': '/c/pt/planta/n/wn/artifact',
                      '@type': 'Node',
                 